### 0. Imports

In [228]:
import pandas as pd
import yfinance as yf
import numpy as np
import xlrd
from itertools import combinations

### 1. Data download

In [148]:
# read ticker data as Series and convert to list
tickers = pd.read_csv('tickers.csv', squeeze=True)
tickers = tickers.tolist()

In [149]:
price_list = []

for ticker in tickers:
    stock = yf.Ticker(ticker).history(period="3mo").reset_index()
    stock = stock[['Date', 'Close']]
    stock.rename(columns={'Close': ticker}, inplace=True)
    price_list.append(stock)

- BFB: No data found, symbol may be delisted


### 2. Compute ratios

In [471]:
# concatenate all prices into one big dataframe
df = pd.concat(price_list, axis=1)
df = df.loc[:,~df.columns.duplicated()].copy()

In [472]:
df.set_index('Date', inplace=True)

In [473]:
# compute all ratios
ratio_df = pd.concat([df[df.columns.difference([col])].div(df.where(df !=0, np.nan)[col], axis=0)\
                           .add_suffix("_" + col) for col in df.columns], axis=1)

### 3. Compute daily price change in %

In [474]:
# compute the % change of the ratio between today and yesterday for the last X days
df_delta = ratio_df.pct_change(periods=1)
df_delta = df_delta.iloc[-4:]

# only select ratios with consistent positive price changes
df_newstocks = df_delta[df_delta.iloc[:] >= 0]
positiveRatiosPercent = df_newstocks.drop(df_newstocks.columns[df_newstocks.apply(lambda col: col.isnull().sum() > 0)], axis=1)
labelList = list(positiveRatiosPercent.columns.values)
df_positiveRatios = ratio_df[labelList]